In [1]:
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Input
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow as tf
import os
import cv2

In [2]:
def depth_to_space(inp, scale):
    return tf.nn.depth_to_space(inp, scale)


def relu6(x):
    return ReLU(max_value=6.0)(x)


def _upscale(inp, inp_filter, scale):
    x = depth_to_space(inp, scale)
    x = Conv2D(inp_filter, (1, 1), strides=(1, 1), padding='same')(x)
    return x


def block(inp, out_filters, exp_ratio):
    channel = K.image_data_format()
    if channel == 'channel_last':
        channel_axis = -1
    else:
        channel_axis = 1
    inp_channel = K.int_shape(inp)[channel_axis]
    exp_filter = inp_channel * exp_ratio
    x = Conv2D(exp_filter, (1, 1), padding='same')(inp)
    x = BatchNormalization()(x)
    x = relu6(x)
    x = DepthwiseConv2D((3, 3), padding='same', strides=(2,2))(x)
    x = relu6(x)
    x = Conv2D(out_filters, (1, 1), padding='same')(x)
    return x


def blocks(inp, out_filt, t, n):
    x = block(inp, out_filt, t)
    for i in range(1, n):
        
        x = block(x, out_filt, t)
    return x


def model(inp_shape):
    inputs = Input(shape=inp_shape)
    x = Conv2D(8, (3, 3), strides=(2, 2), padding='same', name='input_layer')(inputs)
    x = blocks(x, 16, 1, 1)
    x = blocks(x, 32, 6, 1)
    x = blocks(x, 64, 6, 1)
    x = _upscale(x, 32, 2)
    x = _upscale(x, 48, 2)
    x = depth_to_space(x, 4)
    sr_model = Model(inputs, x)
    sr_model.summary()
    return sr_model


In [3]:
# def model(inp_shape):
#     inputs = Input(shape=inp_shape)
#     x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', name='input_layer')(inputs)
#     x = blocks(x, 16, 1, 1, 1)
#     x = blocks(x, 24, 6, 2, 2)
#     x = blocks(x, 32, 6, 3, 2)
#     x = _upscale(x, 32, 4)
#     x = _upscale(x, 48, 4)
#     x = depth_to_space(x, 4)
#     sr_model = Model(inputs, x)
#     sr_model.summary()
#     return sr_model

In [4]:
input_shape = (128, 128, 3)

In [5]:
model = model(input_shape)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
input_layer (Conv2D)         (None, 64, 64, 8)         224       
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        576       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
re_lu (ReLU)                 (None, 64, 64, 64)        0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 32, 32, 64)        640       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 64)        0     

In [22]:
base_pth = "/media/nusrat/New Volume/div2k_dataset/"
val_hr_path = os.path.join(base_pth, "DIV2K_valid_HR/")
val_lr_path = os.path.join(base_pth, "DIV2K_valid_LR_mild/")
train_hr_path = os.path.join(base_pth, "DIV2K_train_HR/")
train_lr_path = os.path.join(base_pth, "DIV2K_train_LR_mild/")
# val_hr_img_names = os.list
# val_lr_img_names
# train_hr_img_names
# train_lr_img_names

In [24]:
file_lines = open("/media/nusrat/New Volume/div2k_dataset/validation.txt", 'r')
Lines = file_lines.readlines()

In [33]:
int(Lines[0].split("_")[-1])

0

In [32]:
Lines[0].split("_")[-1].split("n")

['0\n']

In [16]:
len(list(count.keys()))

100

In [ ]:
import os
import cv2
import keras
import random
import numpy as np
import tensorflow as tf


class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  list_IDs, batch_size=4, dim=(128,128), n_channels=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        
        

        self.list_IDs = list_IDs
        self.n_channels = n_channels
        
        self.shuffle = shuffle
        self.on_epoch_end()
        #print(len(list_IDs))

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        #print(type(indexes))
        #print(indexes)

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))        
        y =np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
#             print(ID)
            try:

                # temp_img = cv2.imread(ID)

                temp_img = []
                temp_img_mask = []
#                 img_names = os.listdir(ID)
#                 file_names = {}
#                 for img_name in img_names:
#                     ind = img_name.split(".")[0]
                    
                    
#                     # file_names["index"] = ind
#                     file_names[ind] = img_name
#                 test_list = [int(i) for i in list(file_names.keys())]
#                 ll = sorted(test_list)
#                 img_list = []
#                 for j in range(len(ll)):
#                     img_list.append(file_names[str(ll[j])])
#                 # print(img_list)
                for im in range(0,45):
                    img = cv2.imread(ID + "/Abs_power/" + str(im)+".jpeg")
#                     print(img.shape)
                    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
#                     dst = cv2.bitwise_and(img, mask)

#                     temp_img_mask.append(dst)
                    temp_img.append(img)
            except:
#                 print(ID)
#                 print(self.labels[ID])
#                 print(self.labels[ID] == 0)
                break
            temp_img = np.array(temp_img)
            temp_img = temp_img/255.0
#             temp_img_mask = np.array(temp_img_mask)
#             temp_img_mask = temp_img_mask/255.0
            data_ind = list(self.m_data.keys())
            for ind in data_ind:
                is_exist = ind in ID
                if is_exist:
                    Metadata[i] = np.float(self.m_data[ind]/255)
                    #print("{} {}".format(ind, ID))            
            X[i] = temp_img
            
            
        #             y[i] = self.labels[ID]
                #             print("{}  {}".format(ID, self.labels[ID]))
                #             print("Heee  yyy ",type(y))
#             if self.labels[ID] == 0:
#                 y.append([1,0])
#             elif self.labels[ID]==1:
#                 y.append([0,1])
            
            
            if self.labels[ID] == 0:
                y.append(0)
                
            elif self.labels[ID]==1:
                y.append(1)
#         print(y)
            
#         for_smooth = tf.keras.utils.to_categorical(y, num_classes=self.n_classes)
        y = np.asarray(y)
        
#         Metadata = Metadata/255.0
#         Metadata = np.asarray(Metadata)
#         print(Metadata)
#         print(y)
#         print(for_smooth)
#         print(y)

       
#         return [X, X_mask], y
        return X, y


In [ ]:
import glob
import pandas as pd
image_names = []

for c in classes:
    
   # x = glob.glob(base_pth+c+"/O1" + "/*")
    x = glob.glob(base_pth + c  + "/*")
    if c == "Amyloid_negative":
        dd = x[0:60]
    else:
        dd = x
    for ss in dd:
        sss = glob.glob(ss+"/*")
        for kk in sss:
            
            image_names = image_names + glob.glob(kk+"/*")
    print(len(image_names))
val_img_names = []

for c in classes:
   # x = glob.glob(base_pth+c+"/O1" + "/*")
    x = glob.glob(val_pth+ c  + "/*")
    if c == "Amyloid_negative":
        dd = x[0:20]
    else:
        dd = x
    for ss in dd:
        sss = glob.glob(ss+"/*")
        for kk in sss:
            
            val_img_names = val_img_names + glob.glob(kk+"/*")
    print(len(val_img_names))

In [34]:

def data_gen(path_to_images, data_type, path_save):
    img_names = os.listdir(path_to_images)
    img_names.sort()
    data = open(path_save + data_type + ".txt", "w")
    for i in range(len(img_names)):
        img = cv2.imread(path_to_images + img_names[i])
        img_h = img.shape[0]
        img_w = img.shape[1]
        count_w = int(img_w / 128)
        count_h = int(img_h / 64)
        for w in range(count_w):
            for h in range(count_h):
                line = img_names[i] + "_"+str(w) + "_" + str(h) + "\n"
                data.write(line)
    data.close()

In [35]:
data_gen(val_hr_path, 'validation_', base_pth)